# Training YOLO with custom dataset, using Ultralytics library
In this notebook, we will train YOLOv11 to detect traffic signs in images.\
For that, we need a dataset with many examples annotated with the corresponding bounding boxes.\
Roboflow is a great platform for constructing a dataset, annotate it, do some data augmentation and mantain dataset versions.\
In this example, I will use my dataset: https://universe.roboflow.com/matyworkspace/just-traffic-signs-45axx \
However, you can create your own dataset with Roboflow and add some augmentations, such as Rotation, Share, Exposure, etc.

## Initial Setup

**IMPORTANT**: ensure GPU is turned on for this notebook. You can do so in Settings > Accelerator > "GPU P100"

### Installing Dependencies
This may take a few minutes...

In [ ]:
!pip install -q ultralytics supervision roboflow

### Setting Parameters

In [ ]:
config = {
    "name": "YOLOv11s",
    "max_epochs": 20,
    "patience": 4, # EarlyStopping: min consecutive epochs with no improvement -> stop training
    "batch_size": 16,
    "object_classes": ['traffic_sign'], # classes of objects in the dataset. in this example we only have one class
    "yolo_version": "yolo11s.pt", # here we can choose the YOLO version. e.g.: "yolov8s.pt". ".pt" versions are pretrained with COCO dataset. More info about versions in: https://docs.ultralytics.com/es/models/yolo11/#supported-tasks-and-modes
}

## Downloading Dataset
When creating a new version of the dataset, in Roboflow we go to "Download Dataset" \
We select "YOLOv8" as Image and Annotation Format (even if we use a newer version of YOLO), and select "Show download code". We can skip the option "Also train a model for Label Assist with Roboflow Train." \
![Generating download code in Roboflow](https://cdn.discordapp.com/attachments/422113807574499339/1433810451677974624/Sin_titulo.png?ex=69060b8a&is=6904ba0a&hm=64863739c48ae167a4a71a7a6de88dae17d962d88e2cf47c13dde54f844565c0&) \
In the Jupyter Tab, you will find the following snippet with the corresponding api_key: \
![](https://cdn.discordapp.com/attachments/422113807574499339/1433810791546617980/image.png?ex=69060bdb&is=6904ba5b&hm=a6c931b76e96b235e9ceb7772a1d75c9d6a9a4482a68f38d476cd2c3f186c2c7&)

In [ ]:
# Code snippet from Roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="API_KEY_GOES_HERE")
project = rf.workspace("matyworkspace").project("just-traffic-signs-45axx")
version = project.version(3)
dataset = version.download("yolov8")
dataset_name = "Just-Traffic-Signs-3"

In [ ]:
# When we create a dataset with Roboflow, it automatically splits between train, valid and test. 
# Since we selected YOLOv8 format, it comes in separated folders.
train_dir = f'/kaggle/working/{dataset_name}/train/'
valid_dir = f'/kaggle/working/{dataset_name}/valid/'
test_dir = f'/kaggle/working/{dataset_name}/test/'

In [ ]:
# We create a YAML to define the dataset for Ultralytics library
data_yaml = f"""
path: /kaggle/working/{dataset_name}
train: train/images
val: valid/images
test: test/images

nc: {len(config['object_classes'])}
names: {config['object_classes']}
"""

with open('/kaggle/working/data.yaml', 'w') as f:
    f.write(data_yaml)

## Training the model

In [ ]:
from ultralytics import YOLO

model = YOLO(config['yolo_version'])
results = model.train(
        data='/kaggle/working/data.yaml',
        epochs=config['max_epochs'],
        imgsz=640,
        patience=config['patience'],
        project='/kaggle/working',
        batch=config['batch_size'],
        name=config['name'],
        cache=True,
        exist_ok=True, # can override previous runs
        optimizer='auto',
        plots=True # generate plots
    )

## Training Results
### Download model

In [ ]:
# Download trained model in Ultralytics format
import os
from IPython.display import FileLink
os.chdir(r'/kaggle/working')
display(FileLink(f"{config['name']}/weights/best.pt"))
# You can click the displayed link to download the file
# then you can load the model again with model = YOLO("file_name.pt")

In [ ]:
# Download in ONNX format
model.export(format="onnx", simplify=True, nms=True)
os.chdir(r'/kaggle/working')
display(FileLink(f"{config['name']}/weights/best.onnx"))

### Training metrics

In [ ]:
from IPython.display import Image, display
display(Image(f"/kaggle/working/{config['name']}/results.png"))

## Testing model

In [ ]:
model.val(data='/kaggle/working/data.yaml', split='test', name=f"{config['name']}_test", plots=True, exist_ok=True)

In [ ]:
name = f"{config['name']}_test"
print(f"--------------------{name}---------------------")
display(Image(f'/kaggle/working/runs/detect/{name}/confusion_matrix.png'))
display(Image(f'/kaggle/working/runs/detect/{name}/val_batch1_pred.jpg'))
display(Image(f'/kaggle/working/runs/detect/{name}/val_batch2_pred.jpg'))
display(Image(f'/kaggle/working/runs/detect/{name}/BoxP_curve.png'))
display(Image(f'/kaggle/working/runs/detect/{name}/BoxR_curve.png'))
display(Image(f'/kaggle/working/runs/detect/{name}/BoxF1_curve.png'))
display(Image(f'/kaggle/working/runs/detect/{name}/BoxPR_curve.png'))